## 读取数据

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import gzip
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """加载MNIST数据集"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte.gz')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte.gz')

    with gzip.open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        struct.unpack('>IIII', imgpath.read(16))
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels),28,28)

    return images, labels



# 数据集划分
def data_split(images, labels, ratio):
    
    total_len = images.shape[0]
    offset = int(total_len * ratio)

    val_img = images[:offset][:]
    val_lb = labels[:offset]

    train_img = images[offset:][:]
    train_lb = labels[offset:]

    return train_img, train_lb, val_img, val_lb    

# 读取训练集和测试集数据
[images, labels] = load_mnist('./MNIST', kind='train')
[test_img, test_lb] = load_mnist('./MNIST',kind='test')
train_img, train_lb, val_img, val_lb = data_split(images, labels, 1/6)

# 为了加快调试速度，从训练集选择2000个样本。
#train_img=train_img[0:2000]

# 将所有数据归一化到0-1之间
train_img =train_img/255.
val_img   =val_img/255.
test_img  =test_img/255.

# 对标签进行热编码
one_hot_train_lb = np.eye(10)[train_lb]
one_hot_val_lb = np.eye(10)[val_lb]
one_hot_test_lb= np.eye(10)[test_lb]

# 打印查看数据集格式
print('训练集图像格式为:', train_img.shape, '训练集标签格式为:', train_lb.shape,'热编码训练集标签格式为:', one_hot_train_lb.shape)
print('验证集图像格式为:', val_img.shape, '验证集标签格式为:', val_lb.shape,'热编码验证集标签格式为:', one_hot_val_lb.shape)
print('测试集图像格式为:', test_img.shape, '测试集标签格式为:', test_lb.shape,'热编码测试集标签格式为:', one_hot_test_lb.shape)

训练集图像格式为: (50000, 28, 28) 训练集标签格式为: (50000,) 热编码训练集标签格式为: (50000, 10)
验证集图像格式为: (10000, 28, 28) 验证集标签格式为: (10000,) 热编码验证集标签格式为: (10000, 10)
测试集图像格式为: (10000, 28, 28) 测试集标签格式为: (10000,) 热编码测试集标签格式为: (10000, 10)


## 数据增强处理

In [2]:
# 程序改错
from torchvision.transforms import v2
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    #v2.RandomVerticalFlip(),
    v2.Normalize(mean=[0.406],std=[0.225]),
    v2.RandomRotation(degrees=(-45, 45))
    ])
train_img=torch.tensor(train_img,dtype=torch.float)
augmented_sample = transforms(train_img)
# 将增强后的数据添加到训练集中
train_img = torch.cat((augmented_sample,train_img),dim=0)
one_hot_train_lb=np.concatenate((one_hot_train_lb,one_hot_train_lb),axis=0)
print(train_img.shape)
print(one_hot_train_lb.shape)

torch.Size([100000, 28, 28])
(100000, 10)


## 使用增强处理后的数据，训练网络

In [3]:
# 程序改错

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1=nn.Linear(28*28, 512)
        self.w2=nn.Linear(512, 512)
        self.w3=nn.Linear(512, 10)
        self.relu=nn.ReLU()
        self.drop=nn.Dropout()

    def forward(self, x):
        x = x.view(len(x),-1)
        x = self.w1 (x)
        x = self.relu(x)
        self.drop(x)
        x = self.w2 (x)
        x = self.relu(x)
        self.drop(x)
        x = self.w3 (x)
        return x

model = NeuralNetwork()

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

batch_size = 200
epochs = 100
batch_num=int(train_img.shape[0]/batch_size)
size = len(train_img)

model.train()
for t in range(epochs):
    
    correct=0.
    train_mean_loss=0.

    for batch in range(batch_num):
        X=train_img[batch*batch_size:(batch+1)*batch_size,]
        y=one_hot_train_lb[batch*batch_size:(batch+1)*batch_size,:]

        X=torch.tensor(X)
        y=torch.tensor(y)
        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()
        train_mean_loss+= loss.item()

    train_mean_loss /= batch_num
    correct /= batch_num
    
    print(f" Epoch:{t}, loss: {train_mean_loss:>8f},  Accuracy: {(100*correct):>0.1f}%")


/tmp/ipykernel_436743/2987195528.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X=torch.tensor(X)


 Epoch:0, loss: 0.271465,  Accuracy: 92.0%
 Epoch:1, loss: 0.135024,  Accuracy: 96.3%
 Epoch:2, loss: 0.090018,  Accuracy: 97.3%
 Epoch:3, loss: 0.063365,  Accuracy: 98.0%
 Epoch:4, loss: 0.048150,  Accuracy: 98.5%
 Epoch:5, loss: 0.042705,  Accuracy: 98.6%
 Epoch:6, loss: 0.035888,  Accuracy: 98.8%
 Epoch:7, loss: 0.032163,  Accuracy: 99.0%
 Epoch:8, loss: 0.029450,  Accuracy: 99.0%
 Epoch:9, loss: 0.029315,  Accuracy: 99.1%
 Epoch:10, loss: 0.020865,  Accuracy: 99.3%
 Epoch:11, loss: 0.024634,  Accuracy: 99.2%
 Epoch:12, loss: 0.018106,  Accuracy: 99.4%
 Epoch:13, loss: 0.019715,  Accuracy: 99.4%
 Epoch:14, loss: 0.018186,  Accuracy: 99.4%
 Epoch:15, loss: 0.015403,  Accuracy: 99.5%
 Epoch:16, loss: 0.016620,  Accuracy: 99.5%
 Epoch:17, loss: 0.017757,  Accuracy: 99.5%
 Epoch:18, loss: 0.017566,  Accuracy: 99.5%
 Epoch:19, loss: 0.013412,  Accuracy: 99.6%
 Epoch:20, loss: 0.010552,  Accuracy: 99.6%
 Epoch:21, loss: 0.015626,  Accuracy: 99.5%
 Epoch:22, loss: 0.014650,  Accuracy: 99.6

In [4]:
model.eval()
test_loss, correct = 0., 0.
with torch.no_grad():
        X=torch.tensor(test_img, dtype=torch.float32)
        y=torch.tensor(one_hot_test_lb, dtype=torch.float32)
        pred = model(X)
        test_loss = np.mean(loss_fn(pred, y).item())
        correct = (pred.argmax(1) == y.argmax(1)).type(torch.float).mean().item()

print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")


Test Accuracy: 98.4%, Test Avg loss: 0.151334 

